In [1]:
# The following section is used to generate information about the datasets so these 
# variables can be initialised in the CNN Model. The information gathered here will
# also be used to determine an appropiate size of the training and testing datasets.
import os                                                   # Import os to change the information surrounding the enviroment 
from keras.preprocessing.image import ImageDataGenerator    # Create tensorflow image batches using data augmentation.
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
locationOfDataset = '/home/dmr/DatasetForExperiments/adware/'
batches = ImageDataGenerator().flow_from_directory(directory=locationOfDataset, target_size=(64,64), batch_size=10000)

# Print information about Images 
imgs, labels = next(batches)
print("Number of Images and Size of Images: ",imgs.shape) # Images with width x length x depth
print("Number of Images and Families: ",labels.shape) # Lablels with batch_size, number of classes

2023-05-05 12:58:23.539006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 107 images belonging to 10 classes.
Number of Images and Size of Images:  (107, 64, 64, 3)
Number of Images and Families:  (107, 10)


In [2]:
# This section will cover the necessary steps used to split the scareware dataset
# into training and testing to evaluate the effectiveness of this convolutional neural 
# network model based on the scareware samples.
import numpy as np                                   # Provides the ability to use numerical data to convert the .dex files to greyscale images
import scipy as sp                                   # Provides the use of algorithms to be used 
from PIL import Image                                # Allow for processing of images

# Splitting the Dataset into Training and testing with a test size of 30% of overall 
# samples in the dataset. 
from sklearn.model_selection import train_test_split    # Used to split the dataset

# Split up the data set, 30% testing 70% training
trainX, testX, trainY, testY = train_test_split(imgs/255.,labels, test_size=0.3)

# Print the information regarding the training and testing sample sizes
#print("Size of Training Set: ", trainX.shape)                   # Print training set information
#print("Size of Testing Set: ",testX.shape)                      # Print testing set information
print("Number of Samples in Training Set: ",trainY.shape)   # Print training set information
print("Number of Samples in Testing Set: ",testY.shape)     # Print testing set information

Number of Samples in Training Set:  (74, 10)
Number of Samples in Testing Set:  (33, 10)


In [3]:
# The Convolutional Neural Network Model
# This section will encompass the aspects surrounding the convultional neural network
# which was used for the experimentation of this research. 
import keras                                            # Import keras
import tensorflow                                       # Import tensorflow
from keras.models import Sequential, Input, Model       # Used for CNN model regarding Models
from keras.layers import Dense, Dropout, Flatten        # Used for CNN model regarding Layers
from keras.layers import Conv2D, MaxPooling2D           # Used for CNN model regarding Layers
from tensorflow.keras.layers import BatchNormalization  # Layer used to normalise inputs
from tensorflow.keras.metrics import AUC                # Used estimate to Area Under Curve

noOfFamilies = 10 # Number of Families in the Sample

# Function used to create the layers of the convolutional neural network model. These parameters were
# constantly changed throughout testing in order to obtain the best possible results. Within this model 
# there are 10 main layers and each layers parameters are explained in detail. 
def CNNModel():
    CNNModel = Sequential()        # Sequential Model is used the experiments

    # Convolutional Layer: With relu Activation, 30 filters, and a kernel size of 3 x 3
    CNNModel.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))                # Layer 1 

    # Max Pooling Layer: With a pool size of 2 x 2
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 2
    
    # Convolutional Layer: 15 filters and kernel size of 3 x 3 
    CNNModel.add(Conv2D(15, (3, 3), activation='relu'))     # Layer 3
    
    # Max Pooling Layer: Pool size of 2 x 2 
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 4
    
    # Drop Out Layer: This layer used to drop 25% of neurons in the model .
    CNNModel.add(Dropout(0.25))                             # Layer 5
    
    # Flatten Layer
    CNNModel.add(Flatten())                                 # Layer 6
    
    # Dense/Fully Connected Layer : 128 Neurons using the relu activation function
    CNNModel.add(Dense(128, activation='relu'))             # Layer 7
    
    # Drop Out Layer: This layer used to drop 50% of neurons in the model.
    CNNModel.add(Dropout(0.5))                              # Layer 8
    
    # Dense/Fully Connected Layer : 50 Neurons using the relu activation function
    CNNModel.add(Dense(50, activation='relu'))              # Layer 9
    
    # Dense/Fully Connected Layer : number of families neurons, using the softmax activation function
    CNNModel.add(Dense(noOfFamilies, activation='softmax')) # Layer 10
    
    CNNModel.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return CNNModel     # Return the Convolutional Neural Network Model

In [4]:
# Measure the effectiveness of the model. This section is used to generate the results of the 
# corresponding convolutional neural network. For the purpose of this mode, there are three main 
# measurements which have to be declared. These are recall, preicision and f1. Listed below
# are the neccessary formulas which are required to obtain these results.
from keras import backend as K

# Recall Measurement
def recallMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(testY, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Precision Measurement
def precisionMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(yPrediction, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# F1 Measurement
def f1Measurement(testY, yPrediction):
    precision = precisionMeasurement(testY, yPrediction)
    recall = recallMeasurement(testY, yPrediction)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
# Initialise the Model
CNNModel = CNNModel()

2023-05-05 12:58:26.603496: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-05 12:58:26.914733: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-05 12:58:26.914813: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-05 12:58:26.916583: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 12:58:26.932454: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-05 12:58:26.9346

In [6]:
# Print the Information of the Model. This includes the information surrounding 
# the layers of the models as well as the number of total parameters
CNNModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 15)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 15)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 15)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2940)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               3

In [7]:
trainY.shape                            # Returns the shape of the trainY array 
trainY_new = np.argmax(trainY, axis=1)  # Converts the trainY array to an array of integers 
trainY_new                              # Returns the array of integer labels

array([9, 8, 7, 0, 1, 0, 3, 2, 5, 7, 0, 2, 3, 9, 0, 3, 3, 2, 6, 4, 7, 3,
       4, 6, 9, 1, 6, 5, 3, 0, 1, 1, 3, 8, 6, 6, 7, 2, 2, 0, 8, 0, 4, 4,
       3, 4, 3, 5, 7, 9, 5, 5, 3, 6, 9, 1, 7, 0, 6, 5, 2, 6, 6, 4, 2, 3,
       1, 7, 4, 3, 4, 1, 2, 2])

In [8]:
# This section of the model imports the class_weight function from the sklearn.utils 
# library. This is used to compute the class weights for a multi-class classification problem.
# The weightOfModel computes the class weights based on the class distribution found in the 
# training set. The class_weight parameter is set to balanced which means that the class 
# weights are automatically adjusted in acorrdance with the information obtained from the 
# input data.

from sklearn.utils import class_weight # Used to compute the class weights for a multi-class classification system
weightOfModel = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(trainY_new),
                                                 y = trainY_new)

# This code is used to map each unique class label in the training set trainY_new variable. 
# This is then uses the class_weight variable initialised above. 
weightOfModel = {l:c for l,c in zip(np.unique(trainY_new), weightOfModel)}

In [9]:
# This is used to compile the Convolutional Neural network Model. It uses the adam optimiser, 
# computes loss using binary_crossentropy and then calculates the measurements listed before.
CNNModel.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=[f1Measurement,precisionMeasurement, 
                    recallMeasurement,
                    AUC()])

In [10]:
# This section of the model is used to trained the convolutional neural network model. 
# This is achieved by using the fit() method from the Keras API of the TensorFlow library which 
# was imported previously. Here is where the number of epochs are declared for the model also
CNNModel.fit(trainX, trainY, validation_data=(testX, testY), 
                epochs=150,  
                class_weight=weightOfModel)

2023-05-05 12:58:27.086815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/150
3/3 [==============================] - 1s 231ms/step - loss: 0.6645 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc: 0.4436 - val_loss: 0.4464 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.4412
Epoch 2/150
3/3 [==============================] - 0s 42ms/step - loss: 0.4655 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc: 0.4242 - val_loss: 0.3679 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.4625
Epoch 3/150
3/3 [==============================] - 0s 42ms/step - loss: 0.3984 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc: 0.4502 - val_loss: 0.3456 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.5230
Epo

3/3 [==============================] - 0s 51ms/step - loss: 0.2119 - f1Measurement: 0.3951 - precisionMeasurement: 0.8990 - recallMeasurement: 0.2581 - auc: 0.9267 - val_loss: 0.2830 - val_f1Measurement: 0.0857 - val_precisionMeasurement: 0.5000 - val_recallMeasurement: 0.0469 - val_auc: 0.7805
Epoch 27/150
3/3 [==============================] - 0s 42ms/step - loss: 0.2093 - f1Measurement: 0.3824 - precisionMeasurement: 0.7399 - recallMeasurement: 0.2583 - auc: 0.9210 - val_loss: 0.2788 - val_f1Measurement: 0.1190 - val_precisionMeasurement: 0.2500 - val_recallMeasurement: 0.0781 - val_auc: 0.7957
Epoch 28/150
3/3 [==============================] - 0s 42ms/step - loss: 0.1843 - f1Measurement: 0.5310 - precisionMeasurement: 0.7647 - recallMeasurement: 0.4115 - auc: 0.9480 - val_loss: 0.2865 - val_f1Measurement: 0.1163 - val_precisionMeasurement: 0.2273 - val_recallMeasurement: 0.0781 - val_auc: 0.7899
Epoch 29/150
3/3 [==============================] - 0s 45ms/step - loss: 0.1873 - f1Me

Epoch 53/150
3/3 [==============================] - 0s 40ms/step - loss: 0.0697 - f1Measurement: 0.8492 - precisionMeasurement: 0.8730 - recallMeasurement: 0.8281 - auc: 0.9941 - val_loss: 0.3504 - val_f1Measurement: 0.1379 - val_precisionMeasurement: 0.1538 - val_recallMeasurement: 0.1250 - val_auc: 0.8058
Epoch 54/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0750 - f1Measurement: 0.8620 - precisionMeasurement: 0.9204 - recallMeasurement: 0.8115 - auc: 0.9949 - val_loss: 0.3252 - val_f1Measurement: 0.1273 - val_precisionMeasurement: 0.1522 - val_recallMeasurement: 0.1094 - val_auc: 0.7881
Epoch 55/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0818 - f1Measurement: 0.7902 - precisionMeasurement: 0.8360 - recallMeasurement: 0.7492 - auc: 0.9890 - val_loss: 0.3149 - val_f1Measurement: 0.1296 - val_precisionMeasurement: 0.1591 - val_recallMeasurement: 0.1094 - val_auc: 0.7994
Epoch 56/150
3/3 [==============================] - 0s 38ms/step - loss: 

3/3 [==============================] - 0s 39ms/step - loss: 0.0487 - f1Measurement: 0.8553 - precisionMeasurement: 0.8742 - recallMeasurement: 0.8375 - auc: 0.9961 - val_loss: 0.3768 - val_f1Measurement: 0.1228 - val_precisionMeasurement: 0.1400 - val_recallMeasurement: 0.1094 - val_auc: 0.8056
Epoch 80/150
3/3 [==============================] - 0s 40ms/step - loss: 0.0447 - f1Measurement: 0.8907 - precisionMeasurement: 0.9027 - recallMeasurement: 0.8792 - auc: 0.9973 - val_loss: 0.4106 - val_f1Measurement: 0.1228 - val_precisionMeasurement: 0.1400 - val_recallMeasurement: 0.1094 - val_auc: 0.7963
Epoch 81/150
3/3 [==============================] - 0s 44ms/step - loss: 0.0456 - f1Measurement: 0.8981 - precisionMeasurement: 0.9099 - recallMeasurement: 0.8867 - auc: 0.9971 - val_loss: 0.4349 - val_f1Measurement: 0.1207 - val_precisionMeasurement: 0.1346 - val_recallMeasurement: 0.1094 - val_auc: 0.7730
Epoch 82/150
3/3 [==============================] - 0s 42ms/step - loss: 0.0435 - f1Me

3/3 [==============================] - 0s 39ms/step - loss: 0.0321 - f1Measurement: 0.8785 - precisionMeasurement: 0.8943 - recallMeasurement: 0.8638 - auc: 0.9973 - val_loss: 0.3959 - val_f1Measurement: 0.1552 - val_precisionMeasurement: 0.1731 - val_recallMeasurement: 0.1406 - val_auc: 0.7796
Epoch 106/150
3/3 [==============================] - 0s 41ms/step - loss: 0.0401 - f1Measurement: 0.8698 - precisionMeasurement: 0.8698 - recallMeasurement: 0.8698 - auc: 0.9957 - val_loss: 0.4137 - val_f1Measurement: 0.1404 - val_precisionMeasurement: 0.1600 - val_recallMeasurement: 0.1250 - val_auc: 0.7666
Epoch 107/150
3/3 [==============================] - 0s 38ms/step - loss: 0.0422 - f1Measurement: 0.8292 - precisionMeasurement: 0.8474 - recallMeasurement: 0.8120 - auc: 0.9951 - val_loss: 0.4335 - val_f1Measurement: 0.1404 - val_precisionMeasurement: 0.1600 - val_recallMeasurement: 0.1250 - val_auc: 0.7568
Epoch 108/150
3/3 [==============================] - 0s 38ms/step - loss: 0.0349 - f

3/3 [==============================] - 0s 39ms/step - loss: 0.0353 - f1Measurement: 0.8794 - precisionMeasurement: 0.8794 - recallMeasurement: 0.8794 - auc: 0.9967 - val_loss: 0.4924 - val_f1Measurement: 0.1167 - val_precisionMeasurement: 0.1250 - val_recallMeasurement: 0.1094 - val_auc: 0.7619
Epoch 132/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0318 - f1Measurement: 0.9221 - precisionMeasurement: 0.9221 - recallMeasurement: 0.9221 - auc: 0.9972 - val_loss: 0.4822 - val_f1Measurement: 0.1207 - val_precisionMeasurement: 0.1346 - val_recallMeasurement: 0.1094 - val_auc: 0.7426
Epoch 133/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0250 - f1Measurement: 0.9116 - precisionMeasurement: 0.9201 - recallMeasurement: 0.9036 - auc: 0.9983 - val_loss: 0.4839 - val_f1Measurement: 0.1186 - val_precisionMeasurement: 0.1296 - val_recallMeasurement: 0.1094 - val_auc: 0.7643
Epoch 134/150
3/3 [==============================] - 0s 43ms/step - loss: 0.0340 - f

In [11]:
# Evaluates the trained  model on the test data and returns the values of the  metrics
scores = CNNModel.evaluate(testX, testY) 
print("--------------------------------------------------------------------------------------------------------------------")

# Evaluates the trained model on the test data and declares the loss, accuracy 
# f1_score, precision and recall
loss, accuracy, f1_score, precision, recall = CNNModel.evaluate(testX, testY)
print("--------------------------------------------------------------------------------------------------------------------")

# Used to Print the Final CNN Accuracy of the model
print("Final Results for The CNN Model")
print("F1 Accuracy of: {:.2f}".format(scores[1]),'%')    # Print Accuracy of Model
print("With a Loss of: {:.2f}".format(scores[0]),'%')    # Print Loss of Model

2/2 [==============================] - 0s 3ms/step - loss: 0.5607 - f1Measurement: 0.1148 - precisionMeasurement: 0.1207 - recallMeasurement: 0.1094 - auc: 0.7151
--------------------------------------------------------------------------------------------------------------------
2/2 [==============================] - 0s 3ms/step - loss: 0.5607 - f1Measurement: 0.1148 - precisionMeasurement: 0.1207 - recallMeasurement: 0.1094 - auc: 0.7151
--------------------------------------------------------------------------------------------------------------------
Final Results for The CNN Model
F1 Accuracy of: 0.11 %
With a Loss of: 0.56 %
